In [3]:
import os
import sys
import torch

sys.path.append(os.getcwd())
from lossless.component.coolchic import CoolChicEncoderParameter
from lossless.component.coolchic import CoolChicEncoder
from lossless.util.config import (
    args, str_args
)
from lossless.util.parsecli import (
    change_n_out_synth,
    get_coolchic_param_from_args,
)
from lossless.util.image_loading import load_image_as_tensor
from lossless.util.logger import TrainingLogger


image_index = 0
im_path = args["input"][image_index]
im_tensor = load_image_as_tensor(im_path, device="cuda:0")
dataset = im_path.split("/")[-2]

logger = TrainingLogger(
    log_folder_path=args["LOG_PATH"],
    image_name=f"{dataset}_" + im_path.split("/")[-1].split(".")[0],
)
logger.log_result(f"{str_args(args)}")
logger.log_result(f"Processing image {im_path}")

encoder_param = CoolChicEncoderParameter(
    **get_coolchic_param_from_args(args, "residue")
)
encoder_param.set_image_size((im_tensor.shape[2], im_tensor.shape[3]))
encoder_param.layers_synthesis = change_n_out_synth(
    encoder_param.layers_synthesis, args["output_dim_size"]
)
coolchic = CoolChicEncoder(param=encoder_param)
coolchic.to_device("cuda:0")

-35 36
-26
Arguments:
  DATASET_PATH: /home/jakub/ETH/thesis/Cool-Chic/coolchic/../datasets/kodak/
  TEST_WORKDIR: /home/jakub/ETH/thesis/Cool-Chic/coolchic/../coolchic/test-workdir/
  LOG_PATH: /home/jakub/ETH/thesis/Cool-Chic/coolchic/../logs/
  workdir: /home/jakub/ETH/thesis/Cool-Chic/coolchic/../coolchic/test-workdir/
  lmbda: 0.001
  job_duration_min: -1
  print_detailed_archi: False
  print_detailed_struct: False
  start_lr: 0.01
  n_itr: 140000
  n_train_loops: 1
  preset: debug
  layers_synthesis_residue: 48-1-linear-relu,X-1-linear-none,X-3-residual-relu,X-3-residual-none
  arm_residue: 24,2
  arm_image_context_size: 8
  n_ft_per_res_residue: 1,1,1,1,1,1,1
  ups_k_size_residue: 8
  ups_preconcat_k_size_residue: 7
  output_dim_size: 9
  patience: 5000
  schedule_lr: True
  freq_valid: 100
  optimized_module: ['all']
  quantizer_type: softround
  quantizer_noise_type: gaussian
  softround_temperature: (0.3, 0.1)
  noise_parameter: (0.25, 0.1)
  quantize_model: True

Processing 

In [4]:
# print(coolchic.get_flops())
# print(coolchic.flops_str())
with torch.no_grad():
    print(coolchic.get_total_mac_per_pixel())
    print(coolchic.str_complexity())

6163.5703125
| module                            | #parameters or shape   | #flops     |
|:----------------------------------|:-----------------------|:-----------|
| model                             | 0.53M                  | 2.424G     |
|  latent_grids                     |  0.524M                |            |
|   latent_grids.0                  |   (1, 1, 512, 768)     |            |
|   latent_grids.1                  |   (1, 1, 256, 384)     |            |
|   latent_grids.2                  |   (1, 1, 128, 192)     |            |
|   latent_grids.3                  |   (1, 1, 64, 96)       |            |
|   latent_grids.4                  |   (1, 1, 32, 48)       |            |
|   latent_grids.5                  |   (1, 1, 16, 24)       |            |
|   latent_grids.6                  |   (1, 1, 8, 12)        |            |
|  synthesis.layers                 |  2.301K                |  0.875G    |
|   synthesis.layers.0              |   0.384K               |   0.132G   |